# Bước 1: Chuẩn bị và xử lý dữ liệu

In [ ]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec


In [ ]:
# Đọc dữ liệu từ file Excel và giữ lại các cột cần thiết
file_path = 'News.xlsx'  # Đường dẫn tới file Excel
df = pd.read_excel(file_path, usecols=['Bank', 'Link', 'Title', 'Description', 'Time_published', 'Evaluate( Human)'])

# Kiểm tra dữ liệu
print("Dữ liệu đầu vào:")
print(df.head())

Dữ liệu đầu vào:
     Bank                                               Link  \
0  ABBank  https://cafef.vn/abbank-giam-toi-15-nam-lai-su...   
1  ABBank  https://cafef.vn/abbank-va-maybank-tang-cuong-...   
2  ABBank  https://cafef.vn/16-ca-nhan-3-to-chuc-so-huu-g...   
3  ABBank  https://cafef.vn/abbank-dat-558-ty-dong-loi-nh...   
4  ABBank  https://cafef.vn/nhieu-uu-dai-danh-cho-khach-h...   

                                               Title  \
0  ABBank giảm tới 1,5%/năm lãi suất cho vay khác...   
1  Maybank tăng cường quan hệ hợp tác chiến lược,...   
2  16 cá nhân, 3 tổ chức sở hữu gần 67% vốn điều ...   
3  ABBANK đạt 558 tỷ đồng lợi nhuận trước thuế tr...   
4  Nhiều ưu đãi dành cho khách hàng ABBANK nhân s...   

                                         Description         Time_published  \
0  Với mong muốn cùng chung tay tiếp sức, hỗ trợ ...  25-09-2024 - 15:14 PM   
1  Vừa qua tại Hà Nội, Ngân hàng TMCP An Bình (AB...  11-09-2024 - 13:04 PM   
2  Ngân hàng TMCP An Bìn

## 1.1. Làm sạch văn bản

In [ ]:
# Làm sạch văn bản: loại bỏ ký tự đặc biệt, dấu câu và chuyển về chữ thường
def clean_text(text):
    # Kiểm tra xem text có phải là kiểu string hay không, nếu không thì chuyển đổi thành string
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r'[^a-zA-Záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ\s]', '', text)  # Loại bỏ ký tự đặc biệt và số
    text = text.lower()  # Chuyển thành chữ thường
    return text

# Áp dụng hàm làm sạch cho cột 'Title' và 'Description'
df['Title'] = df['Title'].apply(clean_text)
df['Description'] = df['Description'].apply(clean_text)

print("Dữ liệu sau khi làm sạch:")
print(df[['Title', 'Description']].head())

Dữ liệu sau khi làm sạch:
                                               Title  \
0  abbank giảm tới năm lãi suất cho vay khách hàn...   
1  maybank tăng cường quan hệ hợp tác chiến lược ...   
2   cá nhân  tổ chức sở hữu gần  vốn điều lệ ngân...   
3  abbank đạt  tỷ đồng lợi nhuận trước thuế trong...   
4  nhiều ưu đãi dành cho khách hàng abbank nhân s...   

                                         Description  
0  với mong muốn cùng chung tay tiếp sức hỗ trợ n...  
1  vừa qua tại hà nội ngân hàng tmcp an bình abba...  
2  ngân hàng tmcp an bình abbank  abb vừa công bố...  
3  kết thúc quý ii ngân hàng tmcp an bình abbank ...  
4  mừng sinh nhật  năm abbank đã triển khai nhiều...  


## 1.2: Tiền xử lý văn bản
- Tokenization: Chúng ta sẽ tách các câu thành các từ nhỏ hơn.
- Lemmatization/Stemming: Giảm từ về dạng gốc để giảm thiểu sự đa dạng.
- Loại bỏ Stop Words: Loại bỏ các từ không mang nhiều ý nghĩa (ví dụ: “và,” “của,”...).

In [ ]:
#Tạo danh sách stop words tiếng Việt từ một file công khai:

import requests

url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-vi/master/stopwords-vi.txt"
response = requests.get(url)
vietnamese_stopwords = set(response.text.splitlines())


In [ ]:
# Cập nhật mã tiền xử lý văn bản bằng cách sử dụng danh sách vietnamese_stopwords:

# Tải xuống tài nguyên cần thiết từ NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab') # Download the missing punkt_tab resource

# Khởi tạo công cụ lemmatizer
lemmatizer = WordNetLemmatizer()

# Hàm tiền xử lý
def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in vietnamese_stopwords]  # Loại bỏ stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return tokens

# Áp dụng tiền xử lý cho cột 'Title' và 'Description'
df['Title'] = df['Title'].apply(preprocess_text)
df['Description'] = df['Description'].apply(preprocess_text)

print("Dữ liệu sau khi tiền xử lý:")
print(df[['Title', 'Description']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Dữ liệu sau khi tiền xử lý:
                                               Title  \
0  [abbank, giảm, lãi, suất, vay, khách, hàng, ản...   
1  [maybank, tăng, cường, quan, hệ, hợp, tác, chi...   
2  [cá, nhân, tổ, chức, sở, hữu, gần, vốn, lệ, ng...   
3  [abbank, đạt, tỷ, đồng, lợi, nhuận, thuế, hoàn...   
4  [ưu, đãi, dành, khách, hàng, abbank, nhân, sin...   

                                         Description  
0  [mong, chung, tay, tiếp, sức, hỗ, trợ, dân, ph...  
1  [hà, nội, ngân, hàng, tmcp, an, bình, abbank, ...  
2  [ngân, hàng, tmcp, an, bình, abbank, abb, công...  
3  [kết, thúc, quý, ii, ngân, hàng, tmcp, an, bìn...  
4  [mừng, sinh, nhật, abbank, triển, khai, ưu, đã...  


## 1.3 Mã hóa đánh giá

In [ ]:
def encode_evaluation(evaluation):
    # Kiểm tra nếu giá trị là float và chuyển đổi thành string trước khi xử lý
    if isinstance(evaluation, float):
        evaluation = str(evaluation)

    if evaluation.lower() == 'positive':
        return 1
    elif evaluation.lower() == 'negative':
        return 0
    else:
        return 2  # Neutral

# Áp dụng mã hóa
df['Evaluate(Human)'] = df['Evaluate( Human)'].apply(encode_evaluation)

# Hiển thị dữ liệu đã mã hóa
print(df['Evaluate(Human)'].head())

0    1
1    1
2    2
3    1
4    1
Name: Evaluate(Human), dtype: int64


In [ ]:
print(df.head())

     Bank                                               Link  \
0  ABBank  https://cafef.vn/abbank-giam-toi-15-nam-lai-su...   
1  ABBank  https://cafef.vn/abbank-va-maybank-tang-cuong-...   
2  ABBank  https://cafef.vn/16-ca-nhan-3-to-chuc-so-huu-g...   
3  ABBank  https://cafef.vn/abbank-dat-558-ty-dong-loi-nh...   
4  ABBank  https://cafef.vn/nhieu-uu-dai-danh-cho-khach-h...   

                                               Title  \
0  [abbank, giảm, lãi, suất, vay, khách, hàng, ản...   
1  [maybank, tăng, cường, quan, hệ, hợp, tác, chi...   
2  [cá, nhân, tổ, chức, sở, hữu, gần, vốn, lệ, ng...   
3  [abbank, đạt, tỷ, đồng, lợi, nhuận, thuế, hoàn...   
4  [ưu, đãi, dành, khách, hàng, abbank, nhân, sin...   

                                         Description         Time_published  \
0  [mong, chung, tay, tiếp, sức, hỗ, trợ, dân, ph...  25-09-2024 - 15:14 PM   
1  [hà, nội, ngân, hàng, tmcp, an, bình, abbank, ...  11-09-2024 - 13:04 PM   
2  [ngân, hàng, tmcp, an, bình, abbank, a


## 1.4. Chia dữ liệu




In [ ]:
# Lọc ra các dòng có nhãn (cột 'Evaluate( Human)' không bị NaN)
df_labeled = df[df['Evaluate(Human)'].notna()]

# Đảm bảo rằng df_labeled chỉ có 6,000 dòng dữ liệu có nhãn
df_labeled = df_labeled.sample(n=6000, random_state=42)

# Tách các cột đặc trưng (Title, Description) và nhãn (Evaluate( Human))
X = df_labeled[['Title', 'Description']]
y = df_labeled['Evaluate(Human)']

# Chia dữ liệu thành tập huấn luyện và kiểm tra (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Phần dữ liệu không có nhãn để dự đoán sau
df_unlabeled = df[df['Evaluate(Human)'].isna()]

# Kết quả
print("Kích thước dữ liệu huấn luyện:", X_train.shape)
print("Kích thước dữ liệu kiểm tra:", X_test.shape)
print("Số lượng dòng không có nhãn để phân loại sau:", df_unlabeled.shape[0])

Kích thước dữ liệu huấn luyện: (4800, 2)
Kích thước dữ liệu kiểm tra: (1200, 2)
Số lượng dòng không có nhãn để phân loại sau: 0


# Bước 2: Trích Xuất Tính Năng

## 2.1. Tính toán TF-IDF
TF-IDF là một kỹ thuật trích xuất tính năng để đo mức độ quan trọng của một từ trong một văn bản so với toàn bộ tập văn bản

In [ ]:
# Khởi tạo TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Áp dụng TF-IDF cho cột văn bản 'Title' và 'Description'
tfidf_title = tfidf_vectorizer.fit_transform(df['Title'].apply(lambda x: ' '.join(x)))
tfidf_description = tfidf_vectorizer.fit_transform(df['Description'].apply(lambda x: ' '.join(x)))

print("Shape of TF-IDF matrix for Title:", tfidf_title.shape)
print("Shape of TF-IDF matrix for Description:", tfidf_description.shape)

Shape of TF-IDF matrix for Title: (18233, 5009)
Shape of TF-IDF matrix for Description: (18233, 6529)


## 2.2. Chuyển đổi từ ngữ thành vector số sử dụng Word2Vec, GloVe, hoặc BERT

Các mô hình này tạo ra vector nhúng (embedding) để biểu diễn từ ngữ với ngữ cảnh:

### 2.2.1Sử dụng Word2Vec (với gensim)
Word2Vec là một kỹ thuật để chuyển đổi từ thành vector dựa trên ngữ cảnh trong văn bản.

In [ ]:
#pip install gensim

In [ ]:
# # Giả sử bạn đã thực hiện tiền xử lý và có danh sách từ
# titles_tokens = df['Title'].tolist()
# descriptions_tokens = df['Description'].tolist()

# # Tạo model Word2Vec
# model_w2v = Word2Vec(sentences=titles_tokens + descriptions_tokens, vector_size=100, window=5, min_count=1, workers=4)

# # # Lấy vector cho một từ
# # vector = model_w2v.wv['từ_bất_kỳ']
# # print("Vector cho từ 'từ_bất_kỳ':", vector)

### 2.2.2. Sử dụng GloVe
GloVe (Global Vectors for Word Representation) là một kỹ thuật khác để chuyển đổi từ thành vector.

In [ ]:
# import numpy as np

# # Tải GloVe từ file
# def load_glove_model(glove_file):
#     glove_model = {}
#     with open(glove_file, 'r', encoding='utf-8') as f:
#         for line in f:
#             split_line = line.split()
#             word = split_line[0]
#             embedding = np.array(split_line[1:], dtype='float32')
#             glove_model[word] = embedding
#     return glove_model

# glove_model = load_glove_model("glove.6B.100d.txt")  # Thay đổi đường dẫn tới file GloVe

# # # Lấy vector cho một từ
# # vector_glove = glove_model.get('từ_bất_kỳ')
# # print("Vector cho từ 'từ_bất_kỳ':", vector_glove)

### 2.2.3. Sử dụng BERT
BERT (Bidirectional Encoder Representations from Transformers) là một mô hình mạnh mẽ để tạo ra vector từ ngữ trong ngữ cảnh.

Cài đặt Transformers
Cài đặt thư viện Transformers của Hugging Face:

In [ ]:
#pip install transformers torch

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Tải mô hình BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Chuyển đổi từ ngữ thành vector
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    # Lấy embedding từ hidden state cuối cùng
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# # Ví dụ
# embedding = get_bert_embedding("câu văn mẫu")
# print("Vector cho câu 'câu văn mẫu':", embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

# Bước 3: Lựa Chọn và Xây Dựng Mô Hình

## 3.1.  Chọn mô hình phù hợp:

 ### SVM

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Hàm kết hợp văn bản, xử lý cả list trong cột 'Title' và 'Description'
def combine_text(row):
    """Combines 'Title' and 'Description' handling potential lists in both columns."""
    # Chuyển 'Title' thành chuỗi nếu có dạng list hoặc list lồng
    title = " ".join([str(item) for sublist in row['Title'] for item in sublist]) if isinstance(row['Title'], list) and any(isinstance(i, list) for i in row['Title']) else " ".join(row['Title']) if isinstance(row['Title'], list) else str(row['Title'])

    # Chuyển 'Description' thành chuỗi nếu có dạng list
    description = " ".join(row['Description']) if isinstance(row['Description'], list) else str(row['Description'])

    return title + " " + description

# Áp dụng hàm để tạo đặc trưng văn bản kết hợp
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Xây dựng pipeline với TF-IDF và SVM
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Bước chuyển văn bản thành TF-IDF
    ('svm', SVC(kernel='linear', random_state=42))  # Bước phân loại với SVM
])

# Huấn luyện mô hình
pipeline.fit(X_train_combined, y_train)

# Dự đoán trên tập kiểm tra
y_pred = pipeline.predict(X_test_combined)

# Đánh giá mô hình
#test_loss, test_accuracy = model.evaluate(X_test_pad, y_test_encoded) # Change y_test to y_test_encoded
#print("Độ chính xác trên tập kiểm tra:", test_accuracy)
print("Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
print("\nBáo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred))

Độ chính xác trên tập kiểm tra: 0.8658333333333333

Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.70      0.36      0.47        90
           1       0.71      0.61      0.65       170
           2       0.90      0.96      0.93       940

    accuracy                           0.87      1200
   macro avg       0.77      0.64      0.68      1200
weighted avg       0.85      0.87      0.85      1200



### CNN

In [ ]:
# pip install tensorflow keras

#### CNN.1 Chuyển đổi văn bản thành chuỗi số
Để huấn luyện CNN, ta sẽ sử dụng Tokenizer từ Keras để chuyển văn bản thành chuỗi số và pad_sequences để làm cho tất cả các chuỗi có độ dài cố định.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Kết hợp 'Title' và 'Description' thành văn bản duy nhất
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Khởi tạo Tokenizer và fit vào dữ liệu huấn luyện
tokenizer = Tokenizer(num_words=10000)  # Sử dụng 10,000 từ phổ biến nhất
tokenizer.fit_on_texts(X_train_combined)

# Chuyển văn bản thành chuỗi số và làm cho chúng có độ dài cố định
X_train_seq = tokenizer.texts_to_sequences(X_train_combined)
X_test_seq = tokenizer.texts_to_sequences(X_test_combined)

# Đảm bảo tất cả chuỗi có cùng độ dài (ví dụ: maxlen=100)
maxlen = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post')

#### CNN.2 Xây dựng mô hình CNN
Mô hình CNN sẽ bao gồm lớp nhúng (Embedding layer) để chuyển các từ thành vector có kích thước cố định, và sau đó là các lớp tích chập (Conv1D), lớp gộp (MaxPooling1D), và lớp kết nối hoàn toàn (Dense) để phân loại.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Khởi tạo tham số
embedding_dim = 100  # Kích thước của vector nhúng
num_classes = len(y_train.unique())  # Số lớp đầu ra (tùy vào bài toán)

# Xây dựng mô hình CNN
model = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=maxlen),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Sử dụng softmax cho phân loại nhiều lớp
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


#### CNN.3 Huấn luyện mô hình
Huấn luyện mô hình với dữ liệu đã chuẩn bị:

In [ ]:
# Huấn luyện mô hình
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder vào dữ liệu huấn luyện và biến đổi
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)  # Sử dụng cùng LabelEncoder cho dữ liệu kiểm tra


# ... (rest of your code) ...

# Huấn luyện mô hình với y_train_encoded
history = model.fit(X_train_pad, y_train_encoded, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.7437 - loss: 0.7798 - val_accuracy: 0.7729 - val_loss: 0.5821
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7883 - loss: 0.4632 - val_accuracy: 0.8031 - val_loss: 0.4288
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8581 - loss: 0.3320 - val_accuracy: 0.7979 - val_loss: 0.4731
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8742 - loss: 0.2643 - val_accuracy: 0.7958 - val_loss: 0.5266
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8895 - loss: 0.2324 - val_accuracy: 0.7990 - val_loss: 0.6410
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9221 - loss: 0.1886 - val_accuracy: 0.8052 - val_loss: 0.6058
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9411 - loss: 0.1543 - val_accuracy: 0.8062 - val_loss: 0.7921
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9598 - loss: 0.1239 - val_accuracy: 0.8292 - val_loss

#### CNN.4 Đánh giá mô hình
Sau khi mô hình đã được huấn luyện, chúng ta sẽ đánh giá trên tập kiểm tra.

In [ ]:
# Đánh giá trên tập kiểm tra
# Import LabelEncoder if not already imported
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder nếu chưa được khởi tạo
label_encoder = LabelEncoder()

# Fit LabelEncoder vào dữ liệu huấn luyện và biến đổi nếu chưa được thực hiện
y_train_encoded = label_encoder.fit_transform(y_train)

# Sử dụng cùng LabelEncoder cho dữ liệu kiểm tra
y_test_encoded = label_encoder.transform(y_test)

test_loss, test_accuracy = model.evaluate(X_test_pad, y_test_encoded) # Change y_test to y_test_encoded
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8314 - loss: 0.9696
Độ chính xác trên tập kiểm tra: 0.8274999856948853


#### CNN.5 Dự đoán và báo cáo phân loại
Cuối cùng, sử dụng mô hình đã huấn luyện để tạo ra dự đoán và báo cáo phân loại chi tiết.

In [ ]:
from sklearn.metrics import classification_report

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test_pad)
y_pred_classes = y_pred.argmax(axis=1)  # Chuyển xác suất thành nhãn

# Chuyển đổi y_pred_classes về dạng nhãn gốc trước khi tính toán báo cáo
y_pred_classes = label_encoder.inverse_transform(y_pred_classes)

# Báo cáo phân loại
print("Báo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred_classes))

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.52      0.40      0.45        90
           1       0.53      0.53      0.53       170
           2       0.90      0.92      0.91       940

    accuracy                           0.83      1200
   macro avg       0.65      0.62      0.63      1200
weighted avg       0.82      0.83      0.82      1200



### LSTM

In [ ]:
#pip install tensorflow keras

#### LSTM.1 Chuẩn bị dữ liệu cho LSTM
Sử dụng lại phần chuẩn bị dữ liệu đã thực hiện cho mô hình CNN. Ta sẽ sử dụng Tokenizer và pad_sequences từ Keras để chuyển văn bản thành chuỗi số và định dạng chiều dài.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Kết hợp 'Title' và 'Description' thành văn bản duy nhất
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Khởi tạo Tokenizer và fit vào dữ liệu huấn luyện
tokenizer = Tokenizer(num_words=10000)  # Sử dụng 10,000 từ phổ biến nhất
tokenizer.fit_on_texts(X_train_combined)

# Chuyển văn bản thành chuỗi số và làm cho chúng có độ dài cố định
X_train_seq = tokenizer.texts_to_sequences(X_train_combined)
X_test_seq = tokenizer.texts_to_sequences(X_test_combined)

# Đảm bảo tất cả chuỗi có cùng độ dài (ví dụ: maxlen=100)
maxlen = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post')

#### LSTM.2 Xây dựng mô hình LSTM
Mô hình LSTM sẽ bao gồm lớp nhúng (Embedding layer), lớp LSTM, lớp gộp (Dropout), và lớp kết nối hoàn toàn (Dense) để phân loại.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Khởi tạo tham số
embedding_dim = 100  # Kích thước của vector nhúng
num_classes = len(y_train.unique())  # Số lớp đầu ra (tùy vào bài toán)

# Xây dựng mô hình LSTM
model = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=maxlen),
    LSTM(128, return_sequences=False),  # LSTM với 128 đơn vị
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Sử dụng softmax cho phân loại nhiều lớp
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


#### LSTM.3 Huấn luyện mô hình
Huấn luyện mô hình với dữ liệu đã chuẩn bị:

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Fit và transform y_train
y_train_encoded = label_encoder.fit_transform(y_train)

# Huấn luyện mô hình
history = model.fit(X_train_pad, y_train_encoded, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.7279 - loss: 0.7951 - val_accuracy: 0.7729 - val_loss: 0.6842
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7792 - loss: 0.6820 - val_accuracy: 0.7729 - val_loss: 0.6911
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7835 - loss: 0.6792 - val_accuracy: 0.7729 - val_loss: 0.6877
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7718 - loss: 0.6975 - val_accuracy: 0.7729 - val_loss: 0.6838
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7758 - loss: 0.6868 - val_accuracy: 0.7729 - val_loss: 0.6850
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7761 - loss: 0.6852 - val_accuracy: 0.7729 - val_loss: 0.6840
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7740 - loss: 0.6951 - val_accuracy: 0.7729 - val_loss: 0.6834
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7786 - loss: 0.6830 - val_accuracy: 0.7729 - val_

#### LSTM.4 Đánh giá mô hình
Sau khi mô hình đã được huấn luyện, chúng ta sẽ đánh giá trên tập kiểm tra

In [ ]:
# Đánh giá trên tập kiểm tra
# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Fit và transform y_train (đã thực hiện trước đó)
y_train_encoded = label_encoder.fit_transform(y_train)

# Huấn luyện mô hình (đã thực hiện trước đó)
# history = model.fit(X_train_pad, y_train_encoded, epochs=10, batch_size=64, validation_split=0.2)

# ----> Transform y_test using the same LabelEncoder <----
y_test_encoded = label_encoder.transform(y_test)

# Đánh giá trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test_encoded)  # Use y_test_encode
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8041 - loss: 0.6242
Độ chính xác trên tập kiểm tra: 0.7833333611488342


#### LSTM.5 Dự đoán và báo cáo phân loại

In [ ]:
from sklearn.metrics import classification_report

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test_pad)
y_pred_classes = y_pred.argmax(axis=1)  # Chuyển xác suất thành nhãn

# ----> Transform y_pred_classes back to original labels <----
y_pred_classes = label_encoder.inverse_transform(y_pred_classes)

# Báo cáo phân loại
print("Báo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred_classes))

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        90
           1       0.00      0.00      0.00       170
           2       0.78      1.00      0.88       940

    accuracy                           0.78      1200
   macro avg       0.26      0.33      0.29      1200
weighted avg       0.61      0.78      0.69      1200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### PhoBERT

In [ ]:
#pip install transformers torch scikit-learn

#### PhoBert.1 Chuẩn bị dữ liệu
Giống như trước, chúng ta sẽ kết hợp cột "Title" và "Description", và sau đó chuẩn bị dữ liệu cho PhoBERT.


In [ ]:
import pandas as pd

# Kết hợp 'Title' và 'Description' thành văn bản duy nhất
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Đặt nhãn cho dữ liệu
y_train = y_train.values
y_test = y_test.values

#### PhoBert.2 Tải mô hình PhoBERT và Tokenizer
Tải mô hình PhoBERT từ Hugging Face.

In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments

# Tải Tokenizer và Mô hình PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = RobertaForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=len(set(y_train)))

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### PhoBert.3 Mã hóa văn bản
Sử dụng tokenizer để mã hóa văn bản đầu vào thành định dạng mà mô hình có thể hiểu được.

In [ ]:
# Mã hóa dữ liệu
train_encodings = tokenizer(list(X_train_combined), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test_combined), truncation=True, padding=True, max_length=128)

#### PhoBert.4 Tạo Dataset
Tạo dataset cho huấn luyện mô hình.

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tạo dataset cho tập huấn luyện và kiểm tra
train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)

#### PhoBert.5 Huấn luyện mô hình
Thiết lập các tham số huấn luyện và tiến hành huấn luyện mô hình.

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels  # Using encoded labels directly

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Ensure labels is an integer by casting to int before creating the tensor
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item

    def __len__(self):
        return len(self.labels)

# Khởi tạo dataset với nhãn đã mã hóa (Using encoded labels)
train_dataset = CustomDataset(train_encodings, y_train_encoded)
test_dataset = CustomDataset(test_encodings, y_test_encoded)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.495100,0.388791
2,0.384900,0.367574
3,0.290600,0.362839


TrainOutput(global_step=1800, training_loss=0.3728297085232205, metrics={'train_runtime': 502.8883, 'train_samples_per_second': 28.635, 'train_steps_per_second': 3.579, 'total_flos': 873207655084800.0, 'train_loss': 0.3728297085232205, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred  # Lấy logits và nhãn
    predictions = logits.argmax(axis=-1)  # Dự đoán nhãn từ logits
    accuracy = accuracy_score(labels, predictions)  # Tính độ chính xác
    return {'eval_accuracy': accuracy}  # Trả về từ điển với độ chính xác


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder nếu chưa được khởi tạo
label_encoder = LabelEncoder()

# Fit LabelEncoder vào dữ liệu huấn luyện và biến đổi nếu chưa được thực hiện
y_train_encoded = label_encoder.fit_transform(y_train)

# Sử dụng cùng LabelEncoder cho dữ liệu kiểm tra
y_test_encoded = label_encoder.transform(y_test)

# Tạo đối tượng Trainer mới với hàm compute_metrics
trainer = Trainer(
    model=model,  # Mô hình đã huấn luyện
    args=training_args,  # Tham số huấn luyện
    train_dataset=train_dataset,  # Tập huấn luyện
    eval_dataset=test_dataset,  # Tập kiểm tra
    compute_metrics=compute_metrics  # Thêm hàm compute_metrics
)

# Đánh giá mô hình
eval_results = trainer.evaluate()  # Đánh giá mô hình

# In kết quả
print("Loss trên tập kiểm tra:", eval_results['eval_loss'])
print("Độ chính xác trên tập kiểm tra:", eval_results['eval_accuracy'])


Loss trên tập kiểm tra: 0.36283934116363525
Độ chính xác trên tập kiểm tra: 0.8866666666666667


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Khởi tạo LabelEncoder nếu chưa được khởi tạo
label_encoder = LabelEncoder()

# Fit LabelEncoder vào dữ liệu huấn luyện và biến đổi nếu chưa được thực hiện
y_train_encoded = label_encoder.fit_transform(y_train)

# Sử dụng cùng LabelEncoder cho dữ liệu kiểm tra
y_test_encoded = label_encoder.transform(y_test)

# Tạo đối tượng Trainer mới với hàm compute_metrics
trainer = Trainer(
    model=model,  # Mô hình đã huấn luyện
    args=training_args,  # Tham số huấn luyện
    train_dataset=train_dataset,  # Tập huấn luyện
    eval_dataset=test_dataset,  # Tập kiểm tra
    compute_metrics=compute_metrics  # Thêm hàm compute_metrics
)

# Đánh giá mô hình
eval_results = trainer.evaluate()  # Đánh giá mô hình

# In kết quả Loss và Độ chính xác
print("Loss trên tập kiểm tra:", eval_results['eval_loss'])
print("Độ chính xác trên tập kiểm tra:", eval_results['eval_accuracy'])

# Dự đoán trên tập kiểm tra
y_pred = trainer.predict(test_dataset).predictions
y_pred_labels = np.argmax(y_pred, axis=1)

# Convert label_encoder.classes_ to a list of strings
target_names = [str(cls) for cls in label_encoder.classes_]  # Convert to strings

# In báo cáo phân loại chi tiết
print("Báo cáo phân loại:\n", classification_report(y_test_encoded, y_pred_labels, target_names=target_names)) # Use target_names

Loss trên tập kiểm tra: 0.36283934116363525
Độ chính xác trên tập kiểm tra: 0.8866666666666667
Báo cáo phân loại:
               precision    recall  f1-score   support

           0       0.65      0.67      0.66        90
           1       0.68      0.80      0.73       170
           2       0.96      0.92      0.94       940

    accuracy                           0.89      1200
   macro avg       0.76      0.80      0.78      1200
weighted avg       0.89      0.89      0.89      1200



### RNN (Nam)

### RoBERTa (Nam)

### Naive Bayes

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Hàm kết hợp văn bản, xử lý cả list trong cột 'Title' và 'Description'
def combine_text(row):
    """Combines 'Title' and 'Description' handling potential lists in both columns."""
    # Chuyển 'Title' thành chuỗi nếu có dạng list hoặc list lồng
    title = " ".join([str(item) for sublist in row['Title'] for item in sublist]) if isinstance(row['Title'], list) and any(isinstance(i, list) for i in row['Title']) else " ".join(row['Title']) if isinstance(row['Title'], list) else str(row['Title'])

    # Chuyển 'Description' thành chuỗi nếu có dạng list
    description = " ".join(row['Description']) if isinstance(row['Description'], list) else str(row['Description'])

    return title + " " + description

# Áp dụng hàm để tạo đặc trưng văn bản kết hợp
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Xây dựng pipeline với TF-IDF và Naive Bayes
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Bước chuyển văn bản thành TF-IDF
    ('nb', MultinomialNB())  # Bước phân loại với Naive Bayes
])

# Huấn luyện mô hình
pipeline.fit(X_train_combined, y_train)

# Dự đoán trên tập kiểm tra
y_pred = pipeline.predict(X_test_combined)

# Đánh giá mô hình
print("Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
print("\nBáo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred))


Độ chính xác trên tập kiểm tra: 0.7833333333333333

Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        90
           1       0.00      0.00      0.00       170
           2       0.78      1.00      0.88       940

    accuracy                           0.78      1200
   macro avg       0.26      0.33      0.29      1200
weighted avg       0.61      0.78      0.69      1200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Logistics Regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Hàm kết hợp văn bản, xử lý cả list trong cột 'Title' và 'Description'
def combine_text(row):
    """Combines 'Title' and 'Description' handling potential lists in both columns."""
    # Chuyển 'Title' thành chuỗi nếu có dạng list hoặc list lồng
    title = " ".join([str(item) for sublist in row['Title'] for item in sublist]) if isinstance(row['Title'], list) and any(isinstance(i, list) for i in row['Title']) else " ".join(row['Title']) if isinstance(row['Title'], list) else str(row['Title'])

    # Chuyển 'Description' thành chuỗi nếu có dạng list
    description = " ".join(row['Description']) if isinstance(row['Description'], list) else str(row['Description'])

    return title + " " + description

# Áp dụng hàm để tạo đặc trưng văn bản kết hợp
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Xây dựng pipeline với TF-IDF và Logistic Regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Bước chuyển văn bản thành TF-IDF
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))  # Sử dụng Logistic Regression
])

# Huấn luyện mô hình
pipeline.fit(X_train_combined, y_train)

# Dự đoán trên tập kiểm tra
y_pred = pipeline.predict(X_test_combined)

# Đánh giá mô hình
print("Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
print("Báo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Độ chính xác trên tập kiểm tra: 0.8533333333333334
Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.71      0.27      0.39        90
           1       0.70      0.51      0.59       170
           2       0.88      0.97      0.92       940

    accuracy                           0.85      1200
   macro avg       0.76      0.58      0.63      1200
weighted avg       0.84      0.85      0.83      1200

Confusion Matrix:
 [[ 24  17  49]
 [  4  87  79]
 [  6  21 913]]


### Random Forest

### XGBoost

In [ ]:
#pip install datasets --upgrade

###